# Anomalous Tropospheric Propagation Analysis using IGRA Radiosonde Data

[IGRA, National Oceanic and Atmospheric Administration](https://www.ncdc.noaa.gov/data-access/weather-balloon/integrated-global-radiosonde-archive)

The Integrated Global Radiosonde Archive (IGRA) consists of radiosonde and pilot balloon observations at over 2,700 globally distributed stations.

In [ ]:
#@title Notebook initialization
# ===================================================================================
# Project:    TropPo 
#             v. 1.0 2020-03-01, ICTP Wireless Lab
# Programmer: Marco Rainone - ICTP Wireless Lab
# Specifications, revisions and verifications:   
#             Marco Zennaro, Ermanno Pietrosemoli, Marco Rainone - ICTP Wireless Lab
# ===================================================================================
#
# The project is released with Mit License
# https://opensource.org/licenses/MIT
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# ===================================================================================
# 
# cell with declaration of libraries and variables
#
# import required modules 
import os
import os.path
import getopt, errno, sys
import datetime
import numpy as np

import pandas as pd
# load pandas table extension for google colab
%load_ext google.colab.data_table

from collections import namedtuple
import matplotlib.pyplot as plt

# instal staticmap library
!pip install staticmap
from staticmap import StaticMap, CircleMarker

import csv, json, sys
import geopy.distance
from array import *

import folium

# ------------------------- mount drive
from google.colab import drive
root_dir = '/content/'
DirData = root_dir + "data/"
print(DirData)
if os.path.isdir(DirData) == False:
  os.mkdir(DirData)
  print(DirData + ' did not exist but was created.')

# ------------------------- variables
#
# IGRA ftp site
RadiosondeDirData = 'ftp://ftp.ncdc.noaa.gov/pub/data/igra/'
RadiosondeDerivedDirData = 'ftp://ftp.ncdc.noaa.gov/pub/data/igra/derived/derived-por/'

# file name of radiosonda index
LstFile = "igra2-station-list.txt"
# local path of radiosonde index downloaded
fpLstFile = DirData + LstFile

# config
#
PathBaseDir = root_dir               # current working directory of the process
access_rights = 0o777                # define the access rights file/folder
csv_sep = ';'                        # char separator for csv

# ------------------------- input data
codeRadioSonda = ""           # input("Radiosonde code ? ")
stationID = ""                # radiosonda code
# filename of radiosonde derived data
# file name igra station log archive
fNameZipIgraLog = ""          # stationID + "-drvd" + ".txt.zip"
fpZipIgraLog = ""             # full path zip igra log

# initialize variables
inpTTNEventsLog = ''
OutDirMap = ''
minDist = 20
flCaseGtwId = True

# ------------------------- mount drive
from google.colab import drive
root_dir = '/content/'
DirData = root_dir + "data/"
print(DirData)
if os.path.isdir(DirData) == False:
  os.mkdir(DirData)
  print(DirData + ' did not exist but was created.')
PathBaseDir = root_dir               # current working directory of the process

# -------------------------------------------------------------------------
# ------------------------- functions

# create full path of file
def full_path(dir, fileName):
  return ('\"' + dir + fileName + '\"')

# unzip zip archive in the same directory
def unZip(dir, zipName):
  fName = full_path(dir, zipName)
  dest = '\"' + dir + '\"'
  !unzip $fName -d $dest

# get file via ftp
def ftpGet(ftpDir, fileName, destDir):
  # ------------------------- ftp get files with wget
  # -P, --directory-prefix=PREFIX save files to PREFIX/
  # -nc, --no-clobber skip downloads that would download to existing files (overwriting them).
  # -N,  --timestamping don't re-retrieve files unless newer than local.
  # -O, --output-document=FILE write documents to FILE.
  # attenzione: -P richiede il path esplicito nell'istruzione, non in una variabile
  src = ftpDir + fileName
  dest = '\"' + destDir + fileName + '\"'
  !wget $src -O $dest

# ------------------------- mount drive
from google.colab import drive
root_dir = '/content/'
# drive.mount(root_dir)
DirData = root_dir + "data/"
print(DirData)
if os.path.isdir(DirData) == False:
  os.mkdir(DirData)
  print(DirData + ' did not exist but was created.')

# -------------------------------------------------------------------------
#
# get full path file or directory
def get_full_path(file_folder_name):
    return (os.path.abspath(file_folder_name))

# check if the full path is file
def is_directory(full_path):
    # os.path.exists checks whether a file or directory exists:
    ris = os.path.exists(full_path)
    if ris:
        # os.path.isdir checks whether it's a directory
        return (os.path.isdir(full_path))
    return False            # the path is not directory

# check if the full path is file
def is_file_name(full_path):
    # os.path.exists checks whether a file or directory exists:
    ris = os.path.exists(full_path)
    if ris:
        # os.path.isdir checks whether it's a directory
        return (not os.path.isdir(full_path))
    return False            # the path is not filename

def get_dir_name(full_path):
    dirname = os.path.dirname(full_path)    # os independent
    return dirname

# return the file name without path
def get_file_name(full_path):
    basename = os.path.basename(full_path)  # os independent
    base = basename.split('.')[0]
    return base

# return the extension of file
def get_file_ext(full_path):
    basename = os.path.basename(full_path)  # os independent
    ext = '.'.join(basename.split('.')[1:])
    if ext == '.':
        return ""
    return ext

def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

# -------------------------------------------------------------------------
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
    
def get_arrows(locations, color='black', size=6, n_arrows=3):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation))
    return arrows
    
# -------------------------------------------------------------------------


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
/content/data/
/content/data/
/content/data/


In [ ]:
#@title Get IGRA radiosonde index
# get via ftp igra radiosonde index
# ------------------------- ftp get files with wget
ftpGet(RadiosondeDirData, LstFile, DirData)

# riga 336
# create pandas dataframe with list of radiosonde
## ------------------------------
## Variable   Columns   Type
## ID            1-11   Character
## LATITUDE     13-20   Real
## LONGITUDE    22-30   Real
## ELEVATION    32-37   Real
## STATE        39-40   Character
## NAME         42-71   Character
## FSTYEAR      73-76   Integer
## LSTYEAR      78-81   Integer
## NOBS         83-88   Integer
## ------------------------------

colIgra2Names = [
    "ICAONAT"   ,   # Character
    "NETCODE"   ,
    "IDCODE"    ,
    "IGRA2_ID"  ,   # Character
    "LATITUDE"  ,   # Real
    "LONGITUDE" ,   # Real
    "ELEVATION" ,   # Real
    "STATE"     ,   # Character
    "NAME"      ,   # Character
    "FSTYEAR"   ,   # Integer
    "LSTYEAR"   ,   # Integer
    "NOBS"      ,   # Integer
]

colIgra2StationList = [
    [ 0, 2],    # ICAONAT   : Character (Icao National Codes)
    [ 2, 3],    # NETCODE   : Character (Network Code: 
                #    I      : ICAO id (last 4 char IGRA2ID), 
                #    M      : WMO id number (last 5 char IGRA2ID),
                #    V      : Vol.Obs.id (last 5 to 6 char IGRA2ID)
                #    W      : WBAN id (last 5 char IGRA2ID)
                #    X      : Special id ("UA" with 6 alpha chr)
    [ 3,11],    # IDCODE    : Integer
    [ 0,11],    # IGRA2_ID  : Character
    [12,20],    # LATITUDE  : Real
    [21,30],    # LONGITUDE : Real
    [31,37],    # ELEVATION : Real
    [38,40],    # STATE     : Character
    [41,71],    # NAME      : Character
    [72,76],    # FSTYEAR   : Integer
    [77,81],    # LSTYEAR   : Integer
    [82,88]     # NOBS      : Integer
]

# https://www.programcreek.com/python/example/101362/pandas.read_fwf
# local path of radiosonde index in csv format
fpCsvFile = DirData + 'igra2station.csv'

igraStation = pd.read_fwf(fpLstFile, names=colIgra2Names, header=None, colspecs=colIgra2StationList)
# igraStation.to_csv('igra2station.csv', header=True, index=True, sep=csv_sep) 
igraStation.to_csv(fpCsvFile, header=True, index=False, sep=csv_sep) 

# ---------------------------------------------------------------
# remove columns: ['alt','datarate','snr','rssi']
delete_columns = ["IDCODE","STATE"]
igraStation.drop(columns=delete_columns, axis=1, inplace=True)

# filter rows with igraStation['LSTYEAR']<act_year
# act_year = int(now.strftime("%Y"))      # now.strftime("%Y-%m-%d %H:%M:%S")
act_year = 2018         # year limit
# filter list: remove lines with LSTYEAR less than act_year
igraStation.drop(igraStation.loc[igraStation['LSTYEAR']<act_year].index, inplace=True)

# filter list: remove lines with coordinates for mobile radiosonde (see igra2-list-format.txt):
# LATITUDE   is the latitude of the station (in decimal degrees, mobile = -98.8888).
# LONGITUDE  is the longitude of the station (in decimal degrees, mobile = -998.8888).
# ELEVATION  is the elevation of the station (in meters, missing = -999.9, mobile = -998.8).
igraStation.drop(igraStation.loc[igraStation['LONGITUDE'] == -998.8888].index, inplace=True)

# reindex
igraStation.reset_index(drop=True, inplace=True)

# -----------------------------------------------------------------
# visualizza il dataframe nella tabella del notebook
# igraStation



--2020-12-04 16:44:59--  ftp://ftp.ncdc.noaa.gov/pub/data/igra/igra2-station-list.txt
           => ‘/content/data/igra2-station-list.txt’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.137, 2610:20:8040:2::137
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.137|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/igra ... done.
==> SIZE igra2-station-list.txt ... 248132
==> PASV ... done.    ==> RETR igra2-station-list.txt ... done.
Length: 248132 (242K) (unauthoritative)

igra2-station-list. 100%[===================>] 242.32K   786KB/s    in 0.3s    

2020-12-04 16:45:00 (786 KB/s) - ‘/content/data/igra2-station-list.txt’ saved [248132]



In [ ]:
# -----------------------------------------------------------------
# show igra radiosonde index
igraStation


,ICAONAT,NETCODE,IGRA2_ID,LATITUDE,LONGITUDE,ELEVATION,NAME,FSTYEAR,LSTYEAR,NOBS
0,AE,M,AEM00041217,24.4333,54.6500,16.0,ABU DHABI INTERNATIONAL AIRPOR,1983,2020,37502
1,AF,M,AFM00040948,34.5500,69.2167,1791.0,KABUL AIRPORT,1961,2020,18915
2,AG,M,AGM00060390,36.6833,3.2167,25.0,DAR-EL-BEIDA,1948,2020,68994
3,AG,M,AGM00060571,31.5000,-2.2500,811.0,BECHAR,1948,2020,58457
4,AG,M,AGM00060630,27.2333,2.5000,273.0,IN-SALAH,1964,2020,49140
...,...,...,...,...,...,...,...,...,...,...
968,VM,M,VMM00048900,10.8167,106.6667,5.0,TAN SON HOA,1954,2020,33635
969,VM,M,VMM00048914,9.1833,105.1500,1.0,CA MAU,1974,2020,8565
970,WA,M,WAM00068110,-22.5667,17.1000,1725.0,WINDHOEK,1986,2020,12545
971,WI,M,WIM00060096,23.7111,-15.9308,8.7,DAKHLA,1974,2020,1636


In [ ]:
#@title Input parameters

# for examples, see:
# https://colab.research.google.com/notebooks/forms.ipynb

#@markdown > Set these parameters for analysis:

#@markdown 1. (latitude, longitude) of first point
#@markdown 1. (latitude, longitude) of second point
#@markdown 1. start date of analysis 
#@markdown 1. n. of days to analyze

#@markdown then press CTRL+F9 (execute All) to start analysis.

#@markdown ---
#@markdown ### Points Coordinates 
#@markdown ![(latitude, longitude)](https://drive.google.com/uc?id=12N37iajZ5Jw4W5i-SfC905GuKyeHbaaz)
#@markdown #### First point: TX node
ID_point_1 = "node1" #@param {type:"string"}
#@markdown > Latitude range from -90 to 90 (-90: South Pole   +90: North Pole)
latitude_1 =  45.9622374#@param {type:"number"}
#@markdown > Longitude range from -180 to 180.
longitude_1 =  13.6671562#@param {type:"number"}

#@markdown #### Second point: RX node or gateway 
ID_point_2 = "gateway" #@param {type:"string"}
#@markdown > Latitude range from -90 to 90 (-90: South Pole   +90: North Pole)
latitude_2 =  45.9354994#@param {type:"number"}
#@markdown > Longitude range from -180 to 180.
longitude_2 =  13.5439247#@param {type:"number"}

#@markdown ---
#@markdown ### Start date and n. days of analysis: 
# Date fields
date_input = '2020-09-16' #@param {type:"date"}
# n. days
n_days =  7#@param {type:"integer"}

if n_days < 1:
  n_days = 1

# print values:
print("Point 1 ({}): ({} , {})".format(ID_point_1, latitude_1, longitude_1))
print("Point 2 ({}): ({} , {})".format(ID_point_2, latitude_2, longitude_2))
print("Start date: {}".format(date_input))
print("n. days: {}".format(n_days))


Point 1 (node1): (45.9622374 , 13.6671562)
Point 2 (gateway): (45.9354994 , 13.5439247)
Start date: 2020-09-16
n. days: 7


In [ ]:
#@title Find the nearest radiosonde

# trova la radiosonda piu' vicina
# vedi rsigra-near.py
# partendo da riga 419

# create an empty dataframe setting columns
col_names =  ['time' , 'distance' , 
	'nodeaddr' , 'lat' , 'lon' , 
	'gwaddr' , 'gtw_lat' , 'gtw_lon' , 
	'rs_id' , 'rs_lat' , 'rs_lon' , 'rs_distance']
data = pd.DataFrame(columns = col_names)

# distance: set all values integer
# ok only in standard python3
# data['rs_distance'] = data['distance'].apply(np.int64)
# all coordinates with 4 decimals:
# see: https://www.geeksforgeeks.org/python-pandas-dataframe-round/
# data = data.round({'rs_lat':4, 'rs_lon':4})
data = data.round({'distance':1, 'lat':4, 'lon':4, 'gwaddr':4, 'gtw_lat':4, 'gtw_lon':4, 'rs_lat':4, 'rs_lon':4, 'rs_distance':1})

data.at[0, 'time'] = pd.to_datetime(date_input)
data.at[0, 'distance'] = np.NaN
data.at[0, 'nodeaddr'] = ID_point_1
data.at[0, 'lat'] = round(latitude_1, 4)
data.at[0, 'lon'] = round(longitude_1, 4)
data.at[0, 'gwaddr'] = ID_point_2
data.at[0, 'gtw_lat'] = round(latitude_2, 4)
data.at[0, 'gtw_lon'] = round(longitude_2, 4)

# column of radiosonde
data['rs_id'] = ID_point_2
data['rs_lat'] = np.NaN
data['rs_lon'] = np.NaN 
data['rs_distance'] = np.NaN 

# p1 & p2
p1_coord = (latitude_1, longitude_1)
p2_coord = (latitude_2, longitude_2)
# distance from p1 & p2
distance=geopy.distance.geodesic(p1_coord, p2_coord).km
data['distance'] = round(distance, 1)

# calculate median point of two coordinates
mlat = (latitude_1 + latitude_2) / 2.0
mlon = (longitude_1 + longitude_2) / 2.0
coords_1 = (mlat, mlon)

# initialize the last_distance to a dummy value, greather than expected
last_distance = 400000.0                    # 400000km
## print(coords_1)
for i2 in igraStation.index:
	coords_2 = (igraStation['LATITUDE'][i2], igraStation['LONGITUDE'][i2])
	distance=geopy.distance.geodesic(coords_1, coords_2).km
	
	# set flag checking value of rs_id is not set (cell has np.NaN)
	if last_distance > distance:
		# set new values in row
		data.at[0, 'rs_id']        = igraStation.at[i2, 'IGRA2_ID']
		data.at[0, 'rs_lat']       = igraStation.at[i2, 'LATITUDE']
		data.at[0, 'rs_lon']       = igraStation.at[i2, 'LONGITUDE'] 
		data.at[0, 'rs_distance']  = round(distance, 1)
		last_distance = distance

# radiosonde = data.rs_id.unique()
radiosonde = data

radiosonde


,time,distance,nodeaddr,lat,lon,gwaddr,gtw_lat,gtw_lon,rs_id,rs_lat,rs_lon,rs_distance
0,2020-09-16 00:00:00,10.0,node1,45.9622,13.6672,gateway,45.9355,13.5439,ITM00016045,45.9806,13.0592,42.5


# Show Map

In [ ]:
#@title
# hidden cell
# modified version of map-rsigra.py
# ----------------------------------------------------------------

# -------------------------------------------------------------------------
# Get command-line arguments

# initialize variables
inpTTNEventsLog = ''
OutDirMap = ''
minDist = 20
flCaseGtwId = True

# ---------------- valori forzati
# usa i dati di questa radiosonda:
# NLM00006260
# inpTTNEventsLog = "23-7-2019.csv"
# print(inpTTNEventsLog)

OutDirMap = DirData
# print(OutDirMap)

# ---------------------------------------------------------------
# in base of inpTTNMapperLog, get file name and extension
# full path inpTTNMapperLog
# fpTTNEventsLog = os.path.join(PathBaseDir, inpTTNEventsLog)
# fpTTNEventsLog = DirData + inpTTNEventsLog

# inpTTNEventsLog = 'map.html'
# fpTTNEventsLog = DirData + inpTTNEventsLog

# get filename of output map file
# OutMapFile = 'map-' + get_file_name(inpTTNEventsLog) + '.html'
# mod 04/12
OutMapFile = 'map.html'

# full path output dir
if not os.path.exists(OutDirMap):
    os.mkdir(OutDirMap, access_rights)
fpOutDir = get_full_path(OutDirMap)     # full path output dir

outDirRadioSonda = fpOutDir

# full path output file
fpOutMapFile = os.path.join(fpOutDir, OutMapFile)
                                                  
# -------------------------------------------------------------------------
# column_names = ['time','distance','nodeaddr','lat','lon','gwaddr','gtw_lat','gtw_lon']
# data = pd.read_csv(fpTTNEventsLog, skipinitialspace = True, sep = csv_sep)
# print(data)

data.drop_duplicates(['nodeaddr','gwaddr'], inplace=True)
# reindex
data.reset_index(drop=True, inplace=True)

# values used from csv:
# lat
# lon
# gtw_lat
# gtw_lon
# rs_lat
# rs_lon

max_lat  = data['lat'].max()
max_lat += data['gtw_lat'].max()
max_lat += data['rs_lat'].max()
min_lat  = data['lat'].min()
min_lat += data['gtw_lat'].min()
min_lat += data['rs_lat'].min()

max_lon  = data['lon'].max()
max_lon += data['gtw_lon'].max()
max_lon += data['rs_lon'].max()
min_lon  = data['lon'].min()
min_lon += data['gtw_lon'].min()
min_lon += data['rs_lon'].min()

ave_lat = (max_lat + min_lat) / 6.0
ave_lon = (max_lon + min_lon) / 6.0

this_map = folium.Map(location=[ave_lat, ave_lon], zoom_start=9)

for i1 in data.index:
    # print("row index: {}...".format(i1))
    
    # radiosonda
    strPop_radiosonda = "'Radiosonde IGRA:<br>{}<br>({:.4f},{:.4f})".format(
                    data.iloc[i1]['rs_id'],
                    data.iloc[i1]['rs_lat'], data.iloc[i1]['rs_lon']
                    )
    this_map.add_child(
        folium.Marker(location=[data.iloc[i1]['rs_lat'], data.iloc[i1]['rs_lon']], 
                    popup= folium.Popup(strPop_radiosonda, sticky=True, show=True),
                    icon=folium.Icon(icon = 'cloud', color='orange', icon_color='white')
                    )
    )
    # device
    strPop_device = "{}<br>({:.4f},{:.4f})".format(
                    data.iloc[i1]['nodeaddr'],
                    data.iloc[i1]['lat'], data.iloc[i1]['lon']
                    )
    this_map.add_child(
        folium.CircleMarker(location=[data.iloc[i1]['lat'], data.iloc[i1]['lon']],
                    fill='true',
                    radius = 10,
                    popup= folium.Popup(strPop_device, sticky=True, show=True),
                    fill_color='red',
                    color = 'clear',
                    fill_opacity=1
                    )
    )
    # gateway
    strPop_gateway = "{}<br>({:.4f},{:.4f})".format(
                    data.iloc[i1]['gwaddr'],
                    data.iloc[i1]['gtw_lat'], data.iloc[i1]['gtw_lon']
                    )
    this_map.add_child(
        folium.CircleMarker(location=[data.iloc[i1]['gtw_lat'], data.iloc[i1]['gtw_lon']],
                    fill='true',
                    radius = 10,
                    popup= folium.Popup(strPop_gateway, sticky=True, show=True),
                    fill_color='blue',
                    color = 'clear',
                    fill_opacity=1
                    )
    )
    # median point 1
    # calculate median point of coordinates from device to gateway
    mlat = (data['lat'][i1] + data['gtw_lat'][i1]) / 2.0
    mlon = (data['lon'][i1] + data['gtw_lon'][i1]) / 2.0
    strPop_median = "Transmission distance:<br>{}km".format(
                    (int(data.iloc[i1]['distance']))
                    )
    this_map.add_child(
        folium.CircleMarker(location=[mlat, mlon],
                    fill='true',
                    radius = 4,
                    popup= folium.Popup(strPop_median, sticky=True, show=True),
                    fill_color='black',
                    color = 'clear',
                    fill_opacity=1
                    )
    )
    # median point 2
    # calculate median point of coordinates from previous median point to radiosonda
    mlat_radio = (mlat + data['rs_lat'][i1]) / 2.0
    mlon_radio = (mlon + data['rs_lon'][i1]) / 2.0
    strPop_median_radio = " radiosonde dist. to trajectory center:<br>{}km".format(
                    (int(data.iloc[i1]['rs_distance']))
                    )
    this_map.add_child(
        folium.CircleMarker(location=[mlat_radio, mlon_radio],
                    fill='true',
                    radius = 2.5,
                    popup= folium.Popup(strPop_median_radio, sticky=True, show=True),
                    fill_color='black',
                    color = 'clear',
                    fill_opacity=1
                    )
    )
    
    # points of line
    points_txrx = [
        [data.iloc[i1]['lat'], data.iloc[i1]['lon']] ,
        [mlat, mlon],
        [data.iloc[i1]['gtw_lat'], data.iloc[i1]['gtw_lon']]
        ]
    # add line
    this_map.add_child(
        folium.PolyLine(points_txrx,
                    # popup= folium.Popup(str(int(data.iloc[i1]['distance'])) + 'km', sticky=True),
                    color="red", weight=2.5, opacity=1)
    )

    points_event = [
        [data.iloc[i1]['lat'], data.iloc[i1]['lon']] ,
        [data.iloc[i1]['gtw_lat'], data.iloc[i1]['gtw_lon']]
        ]
    arrows = get_arrows(locations=points_event, color='black', n_arrows=3)
    for arrow in arrows:
        arrow.add_to(this_map)

    # points of line
    points_b = [
        [data.iloc[i1]['rs_lat'], data.iloc[i1]['rs_lon']] ,
        [mlat, mlon]
        ]
    # add line
    this_map.add_child(
        folium.PolyLine(points_b,
                    # popup= folium.Popup(str(int(data.iloc[i1]['distance'])) + 'km', sticky=True),
                    color="red",  dash_array='10', weight=1.5, opacity=1)
    )
    
    # this_map.add_child(folium.LatLngPopup())

# -------------------------------------------------

# =================================
# SAVE FOLIUM MAP IN HTML FILE
# =================================
#
# ATTENZIONE: 
# VERIFICATO CHE LA MAPPA VA SALVATA PRIMA DI VISUALIZZARLA,
# ALTRIMENTI NON VIENE CREATA LA CELLA CHE VISUALIZZA LA MAPPA NEL NOTEBOOK.
# PROBLEMA DELLA LIBRERIA ???
#
# salva la mappa nel file html
this_map.save(fpOutMapFile)


In [ ]:
#@title
# hidden cell

import folium

# =================================
# SHOW FOLIUM MAP
# =================================
# show map with the IGRA radiosonde nearest to the link
this_map


In [ ]:
#@title get derivative data of radiosonde
# hidden cell

# ===================================================================================
# get derivative data of radiosonde
# ===================================================================================
#
# ------------------------- variables
RadiosondeDerivedDirData = 'ftp://ftp.ncdc.noaa.gov/pub/data/igra/derived/derived-por/'

# ------------------------- input data
codeRadioSonda = 	data.at[0, 'rs_id']
print("Inserito questo codice: {}".format(codeRadioSonda))
stationID = codeRadioSonda.upper()                    # radiosonda code
# filename of radiosonde derived data
stationID = codeRadioSonda.upper()                    # radiosonda code
# file name igra station log archive
fNameZipIgraLog = stationID + "-drvd" + ".txt.zip"
fpZipIgraLog    = DirData + fNameZipIgraLog    # full path to save zip

# ------------------------- ftp get files with wget
# ftpGet(RadiosondeDirData, LstFile, DirData)
ftpGet(RadiosondeDerivedDirData, fNameZipIgraLog, DirData)

# ------------------------- unzip Igra log file
# originale:
# unZip(DirData, fNameZipIgraLog)
# for google collab
!unzip -o $fpZipIgraLog -d $DirData


Inserito questo codice: ITM00016045
--2020-12-04 16:45:01--  ftp://ftp.ncdc.noaa.gov/pub/data/igra/derived/derived-por/ITM00016045-drvd.txt.zip
           => ‘/content/data/ITM00016045-drvd.txt.zip’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.137, 2610:20:8040:2::137
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.137|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/igra/derived/derived-por ... done.
==> SIZE ITM00016045-drvd.txt.zip ... 8387076
==> PASV ... done.    ==> RETR ITM00016045-drvd.txt.zip ... done.
Length: 8387076 (8.0M) (unauthoritative)

ITM00016045-drvd.tx 100%[===================>]   8.00M  2.89MB/s    in 2.8s    

2020-12-04 16:45:05 (2.89 MB/s) - ‘/content/data/ITM00016045-drvd.txt.zip’ saved [8387076]

Archive:  /content/data/ITM00016045-drvd.txt.zip
  inflating: /content/data/ITM00016045-drvd.txt  


In [ ]:
#@title Graph generation

# ===================================================================================
# derived from graph-rsigra-interval.py
# ===================================================================================
#
import gc               # garbage collector
import time
from zipfile import ZipFile
from calendar import timegm
import datetime
import plotly.graph_objs as go
import plotly.express as px

#
def igraDrvdExtract(dirIgraLog, stationID):
    # estrai dall'archivio zip il log igra
    # file name igra log archive
    fNameZipIgraLog = stationID + "-drvd" + ".txt.zip"
    fpZipIgraLog    = os.path.join(dirIgraLog, fNameZipIgraLog)     # full path zip

    with ZipFile(fpZipIgraLog, 'r') as zipObj:
       # Extract all the contents of zip file in dirIgraLog
       zipObj.extractall(dirIgraLog)

# Extract the record time string from line of derived record
# return:
# string formatted time
# epoch time in seconds
#
def strDrvdRecordTime(line, yearLimit):
    # get string fields
    year  = line[13:17]             # year
    month = line[18:20]             # month
    day   = line[21:23]             # day
    hour  = line[24:26]             # hour
    # string date / time
    date_time_str  = year                # year
    date_time_str += '-' + month         # month
    date_time_str += '-' + day           # day
    date_time_str += ' ' + hour          # hour
    date_time_str += ':00:00'
    # if int(year)<1970:
    if int(year)<yearLimit:
        return(date_time_str, 0)
    # date after or equal yearLimit (standard: 1970)
    # date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    utc_time = time.strptime(date_time_str, "%Y-%m-%d %H:%M:%S")
    epoch_time = timegm(utc_time)
    return(date_time_str, epoch_time)
           
# from time string return time in "%Y%m%d%H%M%S", used to create filename
def time_compact(date_time_str):
    utc_time = time.strptime(date_time_str, "%Y-%m-%d %H:%M:%S")
    ris = time.strftime("%Y%m%d%H%M%S", utc_time)
    return(ris)

def igraDrvdCreateIndex(dirIgraLog, stationID, yearLimit=2015):
    keySearch = "#" + stationID             # esempio: #TSM00060760
    print(keySearch)
    
    # file name igra log archive, without extension
    fNameIgraLog = stationID + "-drvd"
    # file name radiosonda log index
    fNameIgraIndex = fNameIgraLog
    fNameIgraIndex += '.idx'
    # file name radiosonda log
    fNameIgraLog += '.txt'
    fpIgraLog = os.path.join(dirIgraLog, fNameIgraLog)
    print(fpIgraLog)
    fpIgraIndex = os.path.join(dirIgraLog, fNameIgraIndex)
    print(fpIgraIndex)

    fIdx = open(fpIgraIndex, 'w')               # open file index to write

    # -------------------- write header
    lineIdx = "date" + csv_sep + "tm_epoch" + csv_sep + "pos_header" + csv_sep + "pos_data"  + csv_sep + "n_rec" + '\n'
    fIdx.write(lineIdx)

    # read file log Igra
    numHeader = -1          # n. row header con data/ora specificata
    numRecords = 0          # n. records for each section

    with open(fpIgraLog,'r') as rsLog:
        rsLog.seek(0, os.SEEK_END)  # go to the end of the file.
        eof = rsLog.tell()          # get the end of file location
        rsLog.seek(0, os.SEEK_SET)  # go to the beginning of the file.
        while(rsLog.tell() != eof):
            pos = rsLog.tell()                  # posizione file prima della lettura riga
            line = rsLog.readline().strip()
            if line.startswith(keySearch)==False:
                numRecords +=1
                continue
            # -------------- new header
            # extract time acquisition
            date_acq, epoch_time = strDrvdRecordTime(line, yearLimit)
            # verify if time before yearLimit
            if epoch_time == 0:
                # record time before limit
                # print("!!! Time before {}:[{}]".format(yearLimit, date_acq))
                numRecords = 0          # n. records for each section    
                posStartData = rsLog.tell()             # posizione file dopo della lettura riga
                continue
            # epoch time in limit
                
            # -------------- time new header is correct
            elif numHeader >= 0:
                # save numRecords and reinit value
                if numHeader == 0:
                    numRecords = 0
                lineIdx = csv_sep + str(numRecords) + '\n'
                fIdx.write(lineIdx)
                numRecords = 0          # n. records for each section    
            numHeader += 1                  # incrementa n. row header con data/ora specificata

            # -------------- init line to write in index file
            # lineIdx = str(numHeader)
            lineIdx = ""
            
            # create record to write csv
            # lineIdx += csv_sep + '\"' + date_acq + '\"'
            lineIdx += date_acq                     # date, time string
            lineIdx += csv_sep + str(epoch_time)    # epoch time
            
            lineIdx += csv_sep + str(pos)           # position header in file
            posStartData = rsLog.tell()             # posizione file dopo della lettura riga
            lineIdx += csv_sep + str(posStartData)

            fIdx.write(lineIdx)                     # write record in csv

    # save last numRecords value
    lineIdx = csv_sep + str(numRecords) + '\n'
    fIdx.write(lineIdx)
    rsLog.close()
    fIdx.close()
    # ------------------ end igraDrvdCreateIndex

# -------------------------------------------------------------------------
# Get command-line arguments

# initialize variables
# inpZipIgraLog = ''
outDir = ''
strSearchTime = ""
dateSearch = []                 # [year, month, day, hour]
days = n_days

### try:
###     opts, args = getopt.getopt(

strSearchTime = date_input + ' 00:00:00'
# -------------------------------------------------------------------------
# fields of dateSearch
dateSearch.append(strSearchTime[0:4])           # year
dateSearch.append(strSearchTime[5:7])           # month
dateSearch.append(strSearchTime[8:10])          # day
dateSearch.append(strSearchTime[11:13])         # hour
dateSearch.append(strSearchTime[14:16])         # minutes

strDateSearch = ""
for i in range(0, 5):
    strDateSearch += dateSearch[i]
print("dateSearch: [{}][{}]".format(dateSearch, strDateSearch))

# format Date in human format
strDateHuman = ""
strDateHuman += dateSearch[0]
for i in range(1, 3):
    strDateHuman += "/"
    strDateHuman += dateSearch[i]
strDateHuman += " " + dateSearch[3]
strDateHuman += ":" + dateSearch[4]

# -------------------------------------------------------------------------

# fpZipIgraLog    = get_full_path(inpZipIgraLog)     # full path zip
dirZipIgraLog   = get_dir_name(fpZipIgraLog)       # directory zip 
nameZipIgraLog  = get_file_name(fpZipIgraLog)      # file name
# stationID = nameZipIgraLog[0:11]

# get file name index igra 
fNameIdxIgraLog  = nameZipIgraLog + '.idx'
fpIdxIgraLog     = os.path.join(dirZipIgraLog, fNameIdxIgraLog)
print("nameZipIgraLog[{}][{}]".format(nameZipIgraLog, stationID))
print("fpIdxIgraLog[{}]".format(fpIdxIgraLog))

outDir = dirZipIgraLog

# original
# with ZipFile(fpZipIgraLog, 'r') as zipObj:
#   # Extract all the contents of zip file in outDir
#   zipObj.extractall(outDir)
#
# for google collab
!unzip -o $fpZipIgraLog -d $outDir

# -------------------------------------------------------------------------
print("create index ...")
igraDrvdCreateIndex(dirZipIgraLog, stationID)

# get index file
print("... read file indice")
idxLog = pd.read_csv(fpIdxIgraLog, sep = csv_sep)
print("... end read indice")

print("start search time in log ...")

search_time = dateSearch[0]
search_time += '-' + dateSearch[1]
search_time += '-' + dateSearch[2]
search_time += ' ' + dateSearch[3]
search_time += ':' + dateSearch[4] + ':00'
print("search_time: [{}]".format(search_time))
                               
# -------------------------------------------------------------------------
# calcola i limiti dei tempi
# convert start time in epochtime
tmUtc_search = time.strptime(search_time, "%Y-%m-%d %H:%M:%S")
search_tmEpoch = timegm(tmUtc_search)
# calculate end time in epochtime
# sum days in seconds to search_tmEpoch
endSrc_tmEpoch = search_tmEpoch + 86400 * days

# get a column with time_epoch
#### selected_columns = idxLog[["tm_epoch"]]
#### dfSearch["tm_epoch"] = selected_columns.copy()
#### dfSearch["tm_epoch"] = dfSearch["tm_epoch"] - search_tmEpoch

# https://stackoverflow.com/questions/37761238/how-do-i-select-and-store-columns-greater-than-a-number-in-pandas
# see example:
# Minimum (for the b column) for the rows satisfying b > 10 condition
# df.loc[df.b > 10, 'b'].min()
# https://community.dataquest.io/t/pandas-return-row-with-the-maximum-value-of-a-column/258474
# example:
# df.iloc[df['column_name'].idxmax()]
# idxmax() will return the index position of the row with the highest value.
# Then you can use iloc to return the row with that index.
# WARNING: If there are multiple max values this method will only return the first row with the max value.
#

min_val = idxLog.loc[(search_tmEpoch >= idxLog.tm_epoch), 'tm_epoch'].max()
#
max_val = idxLog.loc[(endSrc_tmEpoch >= idxLog.tm_epoch), 'tm_epoch'].max()

delta_sec = search_tmEpoch - min_val

# get all rows from min_val and max_val
risIdx = idxLog[(idxLog['tm_epoch']>=min_val ) & (idxLog['tm_epoch']<=max_val)]

print(risIdx)
print("... end search in log")

# reset index and get data
risIdx.reset_index(drop=True, inplace=True)

# -------------------------------------------------------------------------
# https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
nRows_risIdx = risIdx.shape[0]  # gives number of row count
print("n. righe risIdx: {}".format(nRows_risIdx))

n_rec = risIdx.at[0, 'n_rec']
pos_data = risIdx.at[0, 'pos_data']         # start position in file

# ---------------------------------------
delta_hour = int(delta_sec / 3600)
delta_day = delta_hour / 24
print("Differenza di tempo in ore: {}".format(delta_hour))
if delta_day >= 1:
    printf("Warning: il record radiosonda e' stato acquisito da {} giorni rispetto l'orario fornito in ingresso".format(delta_day)) 

# ---------------------------------------

if n_rec == 0:
    print("No record to analyze")
    sys.exit()

fNameIgraLog = get_file_name(fpZipIgraLog)
fNameIgraLog += '.txt'
fpIgraLog = os.path.join(dirZipIgraLog, fNameIgraLog)
print(fpIgraLog)

# --------------------------------------------------------------------------------------
# get inputstation, create report file names

inputstation = stationID.upper()        # esempio: TSM00060760

# html report file from station
str_days = str(days).zfill(3)
HtmlRep_N_Height = "reNH-{}-{}-{}days.html".format(inputstation, strDateSearch, str_days)
### HtmlRep_M_Height = "reMH-{}-{}-{}days.html".format(inputstation, strDateSearch, str_days)
HtmlRep_N_slope  = "slNH-{}-{}-{}days.html".format(inputstation, strDateSearch, str_days)
### HtmlRep_M_slope  = "slMH-{}-{}-{}days.html".format(inputstation, strDateSearch, str_days)

# full path file name html report
fpHtmlRep_N_Height = os.path.join(dirZipIgraLog, HtmlRep_N_Height)
### fpHtmlRep_M_Height = os.path.join(dirZipIgraLog, HtmlRep_M_Height)
fpHtmlRep_N_slope  = os.path.join(dirZipIgraLog, HtmlRep_N_slope)
## fpHtmlRep_M_slope  = os.path.join(dirZipIgraLog, HtmlRep_M_slope)

keySearch = "#" + inputstation                              # search string
idxKey = 0
keySearch += " " + dateSearch[idxKey]                       # add time field
print(keySearch)

# ---------------------------------------------------------------
# see documentation:
# ftp://ftp.ncdc.noaa.gov/pub/data/igra/igra2-dataset-description.docx
# ftp://ftp.ncdc.noaa.gov/pub/data/igra/derived/igra2-derived-format.txt
# -------------------------------
# Variable        Columns Type  
# -------------------------------
# PRESS           1-  7   Integer
# REPGPH          9- 15   Integer   reported geopotential height (meters).
# CALCGPH        17- 23   Integer   calculated geopotential height (meters)
# TEMP           25- 31   Integer
# TEMPGRAD       33- 39   Integer
# PTEMP          41- 47   Integer
# PTEMPGRAD      49- 55   Integer
# VTEMP          57- 63   Integer
# VPTEMP         65- 71   Integer
# VAPPRESS       73- 79   Integer
# SATVAP         81- 87   Integer
# REPRH          89- 95   Integer
# CALCRH         97-103   Integer
# RHGRAD        105-111   Integer
# UWND          113-119   Integer
# UWDGRAD       121-127   Integer
# VWND          129-135   Integer
# VWNDGRAD      137-143   Integer
# N             145-151   Integer   the refractive index (unitless).
# -------------------------------
# 
# Notes:
# REPGPH 	reported geopotential height (meters). 
#         This value is often not available at significant levels.
# 		
# CALCGPH calculated geopotential height (meters). 
#         The geopotential height has been estimated by applying the hydrostatic balance to
# 		the atmospheric layer between the next lower level with a reported geopotential height and the current level.

# https://stackoverflow.com/questions/41386443/create-pandas-dataframe-from-txt-file-with-specific-pattern

# CalcGph   : calculated geopotential height (meters)
# RefIndex  : the refractive index (unitless).

height_limit = 4000             # max height for analysis

# list of graph traces
grTrace_N_HGHT = []             # traces x:'N', y:'HGHT'
#### grTrace_M_HGHT = []             # traces x:'M', y:'HGHT'
grTrace_HGHT_SLOPE_N_H = []     # traces x:'HGHT', y:'slopeN_H'
### grTrace_HGHT_SLOPE_M_H = []     # traces x:'HGHT', y:'slopeM_H'

# ---------------------------------------------------------------

# per ogni riga del dataframe risIdx, genera i file csv dei report
Item = namedtuple('CalcGph', 'RefIndex')
for rowPos in range(nRows_risIdx):
    items = []                                          # launch acquisition data
    date_launch = risIdx.at[rowPos, 'date']             # date/time launch radiosonda
    pos_data = risIdx.at[rowPos, 'pos_data']            # start position in file log

    # form filename of output csv file
    fNameOutCsv = stationID + "-"
    fNameOutCsv += time_compact(date_launch)
    fNameOutCsv += '.csv'
    fpOutCsv = os.path.join(dirZipIgraLog, fNameOutCsv)
    print(fpOutCsv)

    with open(fpIgraLog,'r') as rsLog:
        rsLog.seek(pos_data, os.SEEK_SET)  # go to the beginning of the file displacement pos_data.
        while True:
            line = rsLog.readline().rstrip()
            if not line:
                # eof
                break
            if line.startswith('#'):
                # new record
                break
            # insert data in dataframe
            CalcGph     = int(line[16:23])              # calculated geopotential height (meters)
            RefIndex    = int(line[144:151])            # N, the refractive index (unitless)
            items.append((CalcGph, RefIndex))
            # check if CalcGph is greater than height_limit
            if CalcGph > height_limit:
                # acquisition limit, based on height
                break
    rsLog.close()

    # sys.exit()

    # --------------------------------------------------------------------------------------

    # create dataframe 
    dtAcq02 = pd.DataFrame.from_records(items, columns=['HGHT', 'N'])
    # dataframe columns:
    # HGHT  : altezza (m) (used: CALCGPH calculated geopotential height (m))
    # N     : the refractive index (unitless)

    # --------------------------------------------------------------------------------------
    # calculate dtAcq02['M'] = dtAcq02['N'] + 0.157 * dtAcq02['HGHT']
    # dtAcq02['M'] = dtAcq02.N + 0.157 * dtAcq02.HGHT

    # --------------------------------------------------------------------------------------
    # calculate difference of 'N' and 'HGHT' columns

    # Calculates the difference of a DataFrame element compared with another element in the DataFrame 
    # (default is the element in the same column of the previous row).
    dtAcq02['deltaN'] = dtAcq02['N'].diff()
    # dtAcq02['deltaM'] = dtAcq02['M'].diff()

    dtAcq02['deltaH'] = dtAcq02['HGHT'].diff()

    # calc dtAcq02['deltaH']/dtAcq02['deltaN']
    dtAcq02['slopeN_H'] = dtAcq02['deltaN'].div(dtAcq02['deltaH'])
    # calc dtAcq02['deltaH']/dtAcq02['deltaM']
    # dtAcq02['slopeM_H'] = dtAcq02['deltaM'].div(dtAcq02['deltaH'])

    # Access a group of rows and columns by label(s) or a boolean array.
    dtAcq02.loc[~np.isfinite(dtAcq02['slopeN_H']), 'slopeN_H'] = np.nan
    # dtAcq02.loc[~np.isfinite(dtAcq02['slopeM_H']), 'slopeM_H'] = np.nan

    # note: slopeN_H must be divided by 1000. For this parameter the height is in km
    dtAcq02['slopeN_H'] = dtAcq02.slopeN_H * 1000
    # dtAcq02['slopeM_H'] = dtAcq02.slopeM_H * 1000

    # Drop the rows even with single NaN or single missing values.
    dtAcq02 = dtAcq02.dropna()

    print(dtAcq02)
    dtAcq02.to_csv(fpOutCsv, header=True, index=True, sep=csv_sep)

    # ---------------------------------------------------------
    # save traces in lists

    # traces x:'N', y:'HGHT'
    grTrace_N_HGHT.append(
        go.Scatter(
            x=dtAcq02['N'], y=dtAcq02['HGHT'],
            # line=dict(width=3,color='green'),
            line=dict(width=3),
            name=date_launch
        )
    )

    # dtAcq02['HGHT'] in km
    dtAcq02.HGHT = dtAcq02['HGHT'] / 1000
    # traces x:'HGHT', y:'slopeN_H'
    grTrace_HGHT_SLOPE_N_H.append(     
        go.Scatter(
            x=dtAcq02['HGHT'], y=dtAcq02['slopeN_H'],
            # line=dict(width=3,color='green'),
            line=dict(width=3),                
            name=date_launch
        )
    )
    
    del items
    del [[dtAcq02]]
    gc.collect()                # garbage collection
    ## ------------------------- end for rowPos in range(nRows_risIdx)

# --------------------------------------------------------------------------------------
# GRAPHS OUTPUT
# --------------------------------------------------------------------------------------
# https://plotly.com/python/v3/figure-labels/
# for symbols, see:
# https://www.w3schools.com/html/html_symbols.asp

fig = go.Figure()

# -------------- COMMON DATA

# name_graph = "Station: " + inputstation + "      Date: " + str(inputyear) + "/" + str(inputmonth) + "/" + str(inputday) + " " + str(inputhour) + ":00"
name_graph = "Station: " + inputstation + "<br>Start date: " + strDateHuman

# default font parameters
def_font=dict(
    family='Arial, monospace',
    size=16,
    color="#000000"                 # black
)

def_tickfont = dict(
        size = 16,
        color = "#000000"               # black
),        

# -------------- create graph slope N - H
GraphLayout = go.Layout(
    title=go.layout.Title(
        text='<b>' + name_graph + '</b>',       # bold
        font=dict(
            family='Arial, monospace',
            size=16,
            color="#000000"                 # black
        ),
        xref='paper',
        x=0
    ),
    xaxis=go.layout.XAxis(
        range=[0, 4],
        tickfont = dict(
                size = 16,
                color = "#000000"               # black
        ),        
        title=go.layout.xaxis.Title(
            text='<b>Height (km)</b>',
            font=dict(
                family='Arial, monospace',
                size=16,
                color="#000000"                 # black
            )
        )
    ),
    yaxis=go.layout.YAxis(
        tickfont = dict(
                size = 16,
                color = "#000000"               # black
        ),        
        title=go.layout.yaxis.Title(
            text='<b>&#916;N/&#916;H, km<sup>-1</sup></b>',
            font=dict(
                family='Arial, monospace',
                size=16,
                color="#000000"                 # black
            )
        )
    )
)

fig = go.Figure(data=grTrace_HGHT_SLOPE_N_H, layout=GraphLayout)

fig.update_layout(plot_bgcolor='rgb(255,255,255)')
fig.update_xaxes(showgrid=True,gridwidth=1, gridcolor='LightPink',showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showgrid=True,gridwidth=1, gridcolor='LightPink',showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_layout()

fig.write_html(fpHtmlRep_N_slope, auto_open=False)



dateSearch: [['2020', '09', '16', '00', '00']][202009160000]
nameZipIgraLog[ITM00016045-drvd][ITM00016045]
fpIdxIgraLog[/content/data/ITM00016045-drvd.idx]
Archive:  /content/data/ITM00016045-drvd.txt.zip
  inflating: /content/data/ITM00016045-drvd.txt  
create index ...
#ITM00016045
/content/data/ITM00016045-drvd.txt
/content/data/ITM00016045-drvd.idx
... read file indice
... end read indice
start search time in log ...
search_time: [2020-09-16 00:00:00]
                     date    tm_epoch  pos_header  pos_data  n_rec
3189  2020-09-16 00:00:00  1600214400    25995630  25995788     49
3190  2020-09-16 12:00:00  1600257600    26003236  26003394     51
3191  2020-09-17 00:00:00  1600300800    26011146  26011304     53
3192  2020-09-17 12:00:00  1600344000    26019360  26019518     57
3193  2020-09-18 00:00:00  1600387200    26028182  26028340     56
3194  2020-09-18 12:00:00  1600430400    26036852  26037010     44
3195  2020-09-19 00:00:00  1600473600    26043698  26043856     50
3196

In [ ]:
#@title
# hidden cell

import folium

# =================================
# SHOW GRAPH
# =================================
# show graphic
fig
